In [1]:
import numpy as np
import matplotlib.pyplot as plt
import detection_utils as utils
from detection_utils import e, m, kB 
from detection_utils import plot, plot_ax, plot_errbar, plot_power_spectrum
from detection_utils import W_to_dBm, dBm_to_W, dB_to_frac, frac_to_dB, voltage_noise_to_dBm
from detection_utils import find_width, moving_avg
from detection_utils import Component, DetectionSetup
from detection_utils import BandPassFilter, SpectrumAnalyzer, Amplifier
from detection_utils import HarmonicElectron, TankCircuit
from detection_utils import HarmoincElectronCoupledTank, ElectronCoupledToTank
from detection_utils import AnharmonicElectronSetup

In [45]:
def T_to_A(T, f): 
    """ 
    Find motional amplitude given temperature and frequency, 
    assuming 1/2*kT = 1/2mw^2 A^2
    
    Parameters
    ------
    T : [K] 
        Temperature 
    f : [Hz] 
        Motional frequency 
    """
    return np.sqrt(kB*T/m/(2*np.pi*f)**2)

class AnharmonicElectron(HarmonicElectron): 
    def __init__(self, n, d, f0, Qe=1e4, T=4, m=m, e=e, name='Electron'): 
        super().__init__(n, d, 2*np.pi*f0, m, e, name) 
        self.r = 1 / (Qe*2*np.pi*f0*self.c)
        self.d = d 
        self.T = T
        self.f0 = f0
        self.n = n

    def set_T(self, T): 
        self.T = T 

    def set_d(self, d): 
        self.d = d
        
    def get_A(self, T=None): 
        if T is not None:
            return T_to_A(T, self.f0)
        else: 
            return T_to_A(self.T, self.f0)

    def get_current(self, T=None, d=None): 
        T = self.T if T is None else T 
        d = self.d if d is None else d
        A = self.get_A(T) 
        return e*A*2*np.pi*self.f0/d*self.n

class PLCQ(TankCircuit): 
    def __init__(self, Q, Z0, f0, T=4, QC=1000): 
        super().__init__(Q, Z0, 2*np.pi*f0, T) 
        self.QL = 1 / (1/Q - 1/QC)

In [46]:
n = 100  # number of electrons
d = 1e-3  # effective distance, in m
Q = 500  # quality factor of tank circuit
Qe = 1e4  # quality factor of the electron
Z0 = 150  # characteristic impedance of the tank circuit, in Ohm
f0 = 150e6  # resonant frequency of the tank circuit, in Hz (NOT angular freq)
T = 300  # temperature of the tank circuit, in K

ele = AnharmonicElectron(n, d, f0, Qe, T) 
tank = PLCQ(Q, Z0, f0, T)

In [50]:
ele.get_current(), ele.get_A()*1e6, ele.l, ele.c*1e18, np.sqrt(ele.l/ele.c)

(1.0803591751796805e-09,
 71.54621173707163,
 0.3548691188543275,
 3.1724116569901986,
 334456265.0335915)

In [40]:
tank.QL, tank.L*1e9, tank.C*1e12

(1000.0, 159.15494309189535, 7.07355302630646)

In [42]:
ele.get_current()*Q*Z0, e**2*Q*Z0/m/d**2/2/np.pi

(8.102693813847604e-07, 336.36684900700226)

In [48]:
np.sqrt(4*kB*T*Q*Z0)

3.525030638164724e-08

In [51]:
4*kB*T

1.6567788e-20